阶乘末尾的0的个数。

In [5]:
# 思路：末尾0的个数取决于数值可以转化成int*10^i，而10又可以分解成2*5
# 假设对n!做因式分解得到：
# n!=1*2*3*...*n
#   =(1^cnt_1)*(2^cnt_2)*(3^cnt_3)*(5^cnt_5)*...
# 只要返回min(cnt_2,cnt_5)即可，实际上，cnt_2>cnt_5，所以只需返回cnt_5
# n!=1*...*5*...*10*...*15*...，阶乘展开式中每隔5个数就会出现一个5的因子
# 只要求出[1,n]中出现的5的倍数，并能拆解成多少个5相乘即可

def trailingZeroes(n):
    res=0
    while n>=5:
        res+=n//5
        n//=5
        
    return res

trailingZeroes(25)

6

Number of Digit One. 给定一个范围[1,n]，求在该范围内所有数字中，数字"1"出现的次数。

In [13]:
# 思路：假设以某位为界将数字分隔成两部分，改为数字作为高位数字的末位，其余作为低位
# 如以c将abcde划分为高位abc和低位de，从低往高逐位扫描，将c置为1，然后其他位可以任取值


def countDigitOne(n):
    cut_num = 1    # 用于分割高低位的除数，1、10、100、...
    cnt = 0

    while cut_num <= n:
        high = n//cut_num
        low = n % cut_num

        if high % 10 > 1:    # >=2时，该位置为1,高低位在范围内随意取值，abced的取值范围为[00100,ab199]
            cnt += (high//10+1)*cut_num
        # ==1时，该位置为1，高位取值范围可分为两部分，[00100,ab100]U[ab100,ab1de]
        elif high % 10 == 1:
            cnt += (high//10)*cut_num+low+1
        else:    # ==0时，高位只能少取一个值，低位还是任取，[00100,a(b-1)199]
            cnt += (high//10)*cut_num

        cut_num *= 10    # 切割点左移一位

    return cnt


countDigitOne(11)

4

Nth Digit. 求连续数字序列中'012345678910111213...'的第n位数字。

In [5]:
# 思路：找规律，一位数为1-9，9个数字；两位数为10-99，90个数字；三位数为100-999，900个数字
# 以此类推，n位数的数字共有9*(10^n)个。根据此规律可找出第n位所在的区间段
# 找到区间段之后在定位数字，最后定位是该数字的哪一位


def findNthDigit(n):
    if n <= 0:
        return -1
    if n < 10:
        return n

    bits = 2
    n -= 10    # 从0开始

    # 先找到位数，i位数区间有9*10^(i-1)个数，共占9*10^(i-1)*i位
    while n > (9*(10**(bits-1))*bits):
        n -= 9*(10**(bits-1))*bits
        bits += 1

    # 定位是哪一个数
    start = 10**(bits-1)    # i位的起始数字
    offset = n//bits    # 关于起始数字的偏移量
    number = start+offset    # 对应的数字
    dig_idx = n % bits    # 对应第几位

    return int(str(number)[dig_idx])

0

Non-decreasing Array. 给定一个数组，判断该数组在最多修改一个数字的条件下变成一个非递减数组。

In [8]:
# 思路：扫描统计nums[i]<nums[i-1]的思路是错的
# 正确思路应为顺序扫描数组，并与其前一位数字做大小判断
# 当当前数字小于前一位时，还要与前前一位作比较
# 如果没有前前一位，或者当前数要大于前前位，则修改前位
# 如果有前前位，且当前数还要小于前前位，则修改当前数


def checkPossibility(nums):
    if len(nums) < 3:
        return True

    mod_chance = 1
    for idx in range(1, len(nums)):    # 顺序扫描
        if nums[idx] < nums[idx-1]:
            if mod_chance == 0:    # 首先判断还有没有修改机会
                return False

            if idx == 1 or nums[idx] >= nums[idx-2]:
                nums[idx-1] = nums[i]
            else:
                nums[idx] = nums[idx-1]

            mod_chance -= 1

    return True

False

Reverse Integer. 翻转数字中的数字位并返回新数字。

In [15]:
# 思路：每次使用取余运算取出原数字最后一位，再整除10去掉最后一位
# 对于返回值，每次都需要乘10，然后再加上原数字的最后一位
# 最后添加符号
# 注意，本体最初为编译型语言提供，暗含了溢出的判断

def reverse(x):
    if x>=0:
        sign=1
    else:
        sign=-1
        
    res=0
    x=abs(x)
    while x>0:
        remain=x%10
        x//=10
        
        res=res*10+remain
        
    return res*sign if res<2**31 else 0

-21

Count Primes. 计数n以内的质数个数。

In [21]:
# 思路：空间换时间，设立一个质数标记数组
# 从2开始，计数值+1，扫描其倍数，并在数组中标记
# 再从3开始，计数值+1，扫描倍数并标记
# 逐个设置起点，如果flags[i]=True，则计数值+1
# 优化。一个合数的所有因子都能在根号n的范围内计算出来

def countPrimes(n):
    if n<2:
        return 0
    
    isPrime=[True]*n
    isPrime[0]=isPrime[1]=False
    
    res=0
    for idx in range(2,int(n**0.5)+1):
        if isPrime[idx]:
            # 语法糖，跳步切片，相当于取倍数
            isPrime[idx*idx:n:idx]=[False]*len(isPrime[idx*idx:n:idx])
                
    return sum(isPrime)

4

Max Points on a Line. 给定一系列平面点的坐标，最多有几个点共线。

In [42]:
# 思路：首先统计点的出现次数，因为可能出现重复点的情况
# 然后采用O(n^2)的算法，每次固定一个点，计算其余所有点相对该点的斜率并计数
# 返回最大计数值
# 实质上还是暴力解法，编写代码时注意逻辑的缜密性即可，不难

def maxPoints(points):
    if len(points)<2:
        return len(points)
    
    # 统计各点的出现次数
    p_cnts = dict()
    for p in points:
        p_cnts[tuple(p)] = p_cnts.get(tuple(p), 0)+1

    # 不重复的点
    uni_p = list(p_cnts.keys())
    nums_uni_p = len(uni_p)
    
    # 如果全是重复点，则直接返回重复点的数量
    if nums_uni_p==1:
        return p_cnts[uni_p[0]]

    res = 0
    for fix_p_idx in range(nums_uni_p-1):    # 最后一个点无需固定，因为其后面没有点了
        k_s = dict()    # 斜率字典，用于记录在某一斜率下与fix_p共线的点数

        for act_p_idx in range(fix_p_idx+1, nums_uni_p):
            dx, dy = uni_p[act_p_idx][0] - \
                uni_p[fix_p_idx][0], uni_p[act_p_idx][1]-uni_p[fix_p_idx][1]
            
            if dx==0:
                k='Inf'
            else:
                k = dy/dx    # 注意此处存在精度问题，待使用最大公约数改进
            
            # 把活动点的个数加入斜率字典
            k_s[k] = k_s.get(k, 0) + \
                p_cnts[uni_p[act_p_idx]]
            
        # 当前最大值为字典中的最大值再加上固定点的数量
        cur_max = max(k_s.values())+p_cnts[uni_p[fix_p_idx]]

        res=max(res,cur_max)
        
    return res

maxPoints([[84,250],[0,0],[1,0],[0,-70],[0,-70],[1,-1],[21,10],[42,90],[-42,-230]])

6

In [29]:
tmp={
    '1':1,
    '2':2,
    '3':2,
}.values()
# list(tmp)
max(tmp)

2